In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

# Project 04: Language Models

### Checkpoint Due Date (Questions 1-4): Friday Nov 19, 11:59 PM

### Due Date:  Monday Nov 29, 11:59 PM

#### Checkpoint Instructions
* The checkpoint requires you to turn in questions 1-4.
* The checkpoint will be graded for approximate correctness: easier than the final tests; harder than the doctests.

# Introduction to Language Models

In this project, you will build *[statistical language models](https://en.wikipedia.org/wiki/Language_model)* using public domain books found on [Project Gutenberg](https://www.gutenberg.org/). Language models attempt to capture the likelihood that a given sequence of words occur in a given "language" (the precise term is "corpus" or "corpora"). Here, "language" is a broad term that, in addition to the normal usage, may mean the language of a particular author or style. Applications of language models are found in areas such as language generation, authorship attribution, auto-complete / word-suggestion, and machine translation.

In this project, you will learn about a simple but effective type of language model (LM) called N-gram models. Given a sentence (that is, a sequence of words $w = w_1\ldots w_n$), you would like to understand the probability that sentence is used: 
$$P(w) = P(w_1,\ldots,w_n)$$
However, sentences are built from words, and the likelihood that a word occurs where it does, depends on the words it follows. This points to using *conditional probability* to understand $P(w)$. That is, we can write:

$$
P(w) = P(w_1,\ldots,w_n) = P(w_1) \cdot P(w_2|w_1) \cdot P(w_3|w_1,w_2) \cdot\ldots\cdot P(w_n|w_1,\ldots,w_{n-1})
$$  
This is also called the *chain rule* for probabilities.  
**Example:** Consider the sentence $w = $ `when I eat pizza, I wipe off the grease.`

$$
P(\mbox{when}) \cdot P(\mbox{I | when}) \cdot P(\mbox{ eat | when I})\cdot P(\mbox{ pizza | when I eat}) \cdot \ldots \cdot P(\mbox{ grease | when I eat pizza, I wipe off the})
$$

That is, the probability that the sentence occurs is the product of the probability that each subsequent word follows the words that came before. For example, the probability $P(\mbox{ pizza | when I eat})$ is pretty high, as pizza is something that you eat; thus, this term would contribute to this sentence having a higher probability of occurrence.

### N-gram language models

In the description above, the probability that any word in the sentence (e.g. 'grease') depends on *every* word that preceded it (e.g. 'when'). However, it's often the case the likelihood a word appears in a sentence is influenced more by *nearby* words. 
* N-gram language models capture this concept that only nearby words matter; they assume the probability a word occurs only depends on the previous $(N−1)$ words. 
* That is, an N-gram model says: $P(w_n|w_1,\ldots,w_{n-1}) = P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$.

**Example: trigram model**

Consider the sentence $w = $ `when I eat pizza, I wipe off the grease.`

$$
P(\mbox{when}) \cdot P(\mbox{I | when}) \cdot P(\mbox{ eat | when I})\cdot P(\mbox{ pizza | I eat}) \cdot \ldots \cdot P(\mbox{ the | wipe off}) \cdot P(\mbox{ grease | off the})
$$
In this example, we see the trigram model doesn't consider the beginning of the sentence when considering the likelihood of the end word 'grease'.

**Definition:** Given the descriptions and definitions above, we define the **N-grams of a text** to be the list of sliding windows of length N.

For Example, the trigrams of the sentence $w$ in the example above are:
```
[('when', 'I', 'eat'), ('I', 'eat', 'pizza'), ... , ('wipe', 'off', 'the'), ('off', 'the', 'grease')]
```

### Computing an N-gram language model

N-gram language models are a collection of conditional probabilities for sequences of words of length $N$. For example, in a trigram model, for every 3-word sequence $w_1, w_2, w_3$, you need to compute:
$$P(w_1,w_2,w_3) = P(w_3 | w_1, w_2) = \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)}$$

Where $C(w_1, w_2, w_3)$ is the number of occurrences of the trigram-sequence $w_1, w_2, w_3$ in the dataset and $C(w_1, w_2)$ is the number of occurrences of the bigram-sequence  $w_1, w_2$ in the dataset.

For a general N-gram model, the conditional probabilities are computed by dividing the counts of N-grams by the (N-1)-grams they follow.


### Tokenizing corpora

Computing the probabilities of a language model from a book requires breaking up the text of book into sequences of words. This process is called *tokenization*. In reality though, the sequences are not made up entirely of words, but rather more general terms called *tokens*. In this project tokens will include not only whole words, but also punctuation and other terms. Below are a few examples of other types of tokens:

* Punctuation like `,.;'`. For example, the period makes sense as a token, as certain words tend to end sentences (i.e. come before a `.` in an N-gram) and begin sentences (i.e. come after a `.` in an N-gram).
* We have special 'START' and 'END' tokens that begin and end every word sequence (in our case, paragraphs of words in a given book). These make sense as tokens, as certain words may tend to begin and end paragraphs. 

It is useful for the tokens used to represent START and END to be single characters that can never be found in the text of the book you use to create our language model. Thus, you will use two ascii hidden "[control characters](https://en.wikipedia.org/wiki/C0_and_C1_control_codes#C0_(ASCII_and_derivatives))":
* For START, you will use the character `\x02` (used for 'beginning of text')
* For END, you will use the character `\x03` (used for 'end of text')


### A note on checking your work for correctness

Building a statistical model involves stringing together difficult-to-follows steps such as data-cleaning and implementing derived mathematical formulas. A fact of life in this line of work is that it can be difficult to assess the correctness of your work. 

* The Doctests will include checks on the inputs/outputs of your methods.
* Doctests will include models on which you can check your work 'by hand'.
* Additionally, **you should run your functions on *real* books and assess the reasonableness of your result** as well. Merely passing the doctests is never sufficient to guarantee correctness of your code.

# Project Outline

Below is an outline of the the project. If there are terms you don't understand in following descriptions, you should review the introduction for an explanation of the terms and ideas described.

1. Write code to download the text of a book from a url, returning the corpus as a string.
1. Tokenize the corpus into sequences of words and paragraphs.
1. Create two "baseline" langunage models; create text with them using sampling.
1. Compute an N-gram language model
1. Write code to sample from the N-gram language model; create text with them.
1. Evaluate performance (computational) of your language model for different N.
1. Evaluate performance (statistical) of your language model for different N.
1. Create an auto-completion suggestion model.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
import re
import requests
import time

In [4]:
from project import *

## Downloading the book(s)

**Question 1**

For this question, you will use `requests` to download and prepare a public domain book from [Project Gutenberg](). Create a function `get_book` that takes in the `url` of a 'Plain Text UTF-8' book and returns a string containing the contents of the book. 

The function should satisfy the following conditions:
* The contents of the book consist of everything between Project Gutenberg's START and END comments.
* The contents *will* include title/author/table of contents.
* You should also transform any Windows new-lines (`\r\n`) with standard new-lines (`\n`).
* You should check the site's `robots.txt` as well and implement a 'pause' in your request in accordance with the website's policy. If the function is called twice in succession, it should not violate the `robots.txt` policy.

*Note:* You are encouraged to find whatever books on the website that interest you to test your code and the language models you develop. The text doesn't even need to be an English-language book.

In [6]:
url = 'http://www.gutenberg.org/files/74/74-0.txt'
book_string = get_book(url)

In [ ]:
def get_book(url):
    """
    get_book that takes in the url of a 'Plain Text UTF-8' book and
    returns a string containing the contents of the book.

    The function should satisfy the following conditions:
        - The contents of the book consist of everything between
        Project Gutenberg's START and END comments.
        - The contents will include title/author/table of contents.
        - You should also transform any Windows new-lines (\r\n) with
        standard new-lines (\n).
        - If the function is called twice in succession, it should not
        violate the robots.txt policy.

    :Example: (note '\n' don't need to be escaped in notebooks!)
    >>> url = 'http://www.gutenberg.org/files/57988/57988-0.txt'
    >>> book_string = get_book(url)
    >>> book_string[:20] == '\\n\\n\\n\\n\\nProduced by Chu'
    True
    """
    request = requests.get(url)
    time.sleep(5)
    text = request.text
    start = "\*\*\* START[\w\s]+PROJECT GUTENBERG EBOOK[\w\s]+ \*\*\*"
    end = "\*\*\* END[\w\s]+PROJECT GUTENBERG EBOOK[\w\s]+ \*\*\*"
    content = re.split(start, re.split(end, text)[0])[1]
    transformed = re.sub('\r\n', '\n', content)
    return transformed

In [ ]:
grader.check("q1")

## Tokenizing the Corpus

**Question 2**

Now you need to tokenize the corpus in `book_string` (turn the text into a list of tokens -- i.e. words and punctuation). More specifically, create a function `tokenize` that takes in `book_string` and outputs a list of tokens satisfying the following conditions:
* The start of any paragraph should be represented in the list with the single character `\x02` (standing for START).
* The end of any paragraph should be represented in the list with the single character `\x03` (standing for STOP).
* Tokens in the sequence of words are split apart at 'word boundaries' (see the regex lecture).
* Tokens should include *no* whitespace.
* Whitespace (e.g. multiple new-lines) between two paragraphs of text should be ignored.
* Two or more newlines count as a paragraph break

For example, the following sentence (the ellipses denote preceding/continuing text):
```
...
My phone's dead.

I didn't get your call.
...
```
should tokenize to:
```
[ ...
"My", "phone", "'", "s", "dead", ".", "\x03", "\x02", "I", "didn", "'", "t", "get", "your", "call", "."
... ]
```

*Remark:* Your tokenization function should run quickly; you should avoid loops when possible. Your `tokenize` function should run on the the complete works of Shakespeare (in `data/shakespeare`) in **under 10 seconds** to get full credit. Be sure to only request the book once (using your `get_book` function) and save it to file for testing!

In [20]:
shakespeare_fp = os.path.join('data', 'shakespeare.txt')
shakespeare = open(shakespeare_fp, encoding='utf-8').read()

In [21]:
# time your code
# %timeit tokenize(shakespeare)

In [ ]:
def tokenize(book_string):
    """
    tokenize takes in book_string and outputs a list of tokens
    satisfying the following conditions:
        - The start of any paragraph should be represented in the
        list with the single character \x02 (standing for START).
        - The end of any paragraph should be represented in the list
        with the single character \x03 (standing for STOP).
        - Tokens in the sequence of words are split
        apart at 'word boundaries' (see the regex lecture).
        - Tokens should include no whitespace.

    :Example:
    >>> test_fp = os.path.join('data', 'test.txt')
    >>> test = open(test_fp, encoding='utf-8').read()
    >>> tokens = tokenize(test)
    >>> tokens[0] == '\x02'
    True
    >>> tokens[9] == 'dead'
    True
    >>> sum([x == '\x03' for x in tokens]) == 4
    True
    >>> '(' in tokens
    True
    """
    token_lst = []
    start = '\x02'
    end = ' \x03'
    combined = ' \x03\n\x02 '

    start_break = '^(\s*)'
    end_break = '(\s+)$'
    combined_break = '(\s\s+)'


    format_start_break = re.sub(start_break, start, book_string)
    format_end_break = re.sub(end_break, end, format_start_break)
    formatted = re.sub(combined_break, combined, format_end_break)
    split = re.split(r'\b|\s', formatted)
    for token in split:
        if token != ' ' and token != '':
            if re.match('\W+', token) != None:
                token_lst.extend(list(token))
            else:
                token_lst.append(token)

    return token_lst

In [ ]:
grader.check("q2")

# Creating a Language Model

Now that you've learned how to (theoretically) build a language model, you have to implement it in code. Every language model you build will be a class with a few methods in common:

* The `__init__` constructor: when you instantiate an LM object, you will need to pass the 'training corpus' on which your model will be trained (i.e. a list of tokens you created above with `tokenize`). The `train` method will then use that data to create a model which is saved in the `mdl` attribute. This code is given to you.
* The `train` method takes in a list of tokens (e.g. the output of `tokenize`) and outputs a language model. This language model is usually represented as a `Series` (indexed by tokens; values are probabilities that token occurs), or a `DataFrame`.
* The `probability` method takes in a sequence of tokens and returns the probability that this sequence occurs under the language model.
* The `sample` method takes in a number `N > 0` and generates a string made up of `N` tokens using the language model. This method generates language.

You will create three language model classes in the problems below (Uniform; Unigram; N-gram). For each of these, you will implement each of these methods.

## Baseline Models: uniform and unigram LMs

### Uniform probability language model

**Question 3**

A uniform language model is one in which any word is equally likely to appear in any position. The starter code contains a class `UniformLM` which represents a uniform language model. Review the specifics of the class methods above (and in the starter-code) and implement the methods.

In [36]:
unif = UniformLM(homer)
unif.sample(100)

In [ ]:
class UniformLM(object):
    """
    Uniform Language Model class.
    """

    def __init__(self, tokens):
        """
        Initializes a Uniform languange model using a
        list of tokens. It trains the language model
        using `train` and saves it to an attribute
        self.mdl.
        """
        self.mdl = self.train(tokens)

    def train(self, tokens):
        """
        Trains a uniform language model given a list of tokens.
        The output is a series indexed on distinct tokens, and
        values giving the (uniform) probability of a token occuring
        in the language.

        :Example:
        >>> tokens = tuple('one one two three one two four'.split())
        >>> unif = UniformLM(tokens)
        >>> isinstance(unif.mdl, pd.Series)
        True
        >>> set(unif.mdl.index) == set('one two three four'.split())
        True
        >>> (unif.mdl == 0.25).all()
        True
        """
        n_unique_tokens = len(set(tokens))
        probs = 1 / n_unique_tokens
        return pd.Series(probs, index=np.unique(tokens))

    def probability(self, words):
        """
        probability gives the probabiliy a sequence of words
        appears under the language model.
        :param: words: a tuple of tokens
        :returns: the probability `words` appears under the language
        model.

        :Example:
        >>> tokens = tuple('one one two three one two four'.split())
        >>> unif = UniformLM(tokens)
        >>> unif.probability(('five',))
        0
        >>> unif.probability(('one', 'two')) == 0.0625
        True
        """
        try:
            prob = 1
            for word in words:
                prob *= self.mdl[word]
            return prob
        except:
            return 0

    def sample(self, M):
        """
        sample selects tokens from the language model of length M, returning
        a string of tokens.

        :Example:
        >>> tokens = tuple('one one two three one two four'.split())
        >>> unif = UniformLM(tokens)
        >>> samp = unif.sample(1000)
        >>> isinstance(samp, str)
        True
        >>> len(samp.split()) == 1000
        True
        >>> s = pd.Series(samp.split()).value_counts(normalize=True)
        >>> np.isclose(s, 0.25, atol=0.05).all()
        True
        """
        return ' '.join(np.random.choice(self.mdl.index, size=M, replace=True))

In [ ]:
grader.check("q3")

### Unigram language model

**Question 4**

A unigram language model is one in which the likelihood a word appears is proportional to its occurrence in the text. That is, it's just the (unconditional) empirical distribution of words in the corpus. The starter code contains a class `UnigramLM` which represents a unigram language model. Review the specifics of the class methods above (and in the starter-code) and implement the methods.

In [ ]:
class UnigramLM(object):

    def __init__(self, tokens):
        """
        Initializes a Unigram languange model using a
        list of tokens. It trains the language model
        using `train` and saves it to an attribute
        self.mdl.
        """
        self.mdl = self.train(tokens)

    def train(self, tokens):
        """
        Trains a unigram language model given a list of tokens.
        The output is a series indexed on distinct tokens, and
        values giving the probability of a token occuring
        in the language.

        :Example:
        >>> tokens = tuple('one one two three one two four'.split())
        >>> unig = UnigramLM(tokens)
        >>> isinstance(unig.mdl, pd.Series)
        True
        >>> set(unig.mdl.index) == set('one two three four'.split())
        True
        >>> unig.mdl.loc['one'] == 3 / 7
        True
        """
        return pd.Series(tokens).value_counts(normalize=True)

    def probability(self, words):
        """
        probability gives the probabiliy a sequence of words
        appears under the language model.
        :param: words: a tuple of tokens
        :returns: the probability `words` appears under the language
        model.

        :Example:
        >>> tokens = tuple('one one two three one two four'.split())
        >>> unig = UnigramLM(tokens)
        >>> unig.probability(('five',))
        0
        >>> p = unig.probability(('one', 'two'))
        >>> np.isclose(p, 0.12244897959, atol=0.0001)
        True
        """
        try:
            prob = 1
            for word in words:
                prob *= self.mdl[word]
            return prob
        except: # look into what type of error should go here
            return 0

    def sample(self, M):
        """
        sample selects tokens from the language model of length M, returning
        a string of tokens.

        >>> tokens = tuple('one one two three one two four'.split())
        >>> unig = UnigramLM(tokens)
        >>> samp = unig.sample(1000)
        >>> isinstance(samp, str)
        True
        >>> len(samp.split()) == 1000
        True
        >>> s = pd.Series(samp.split()).value_counts(normalize=True).loc['one']
        >>> np.isclose(s, 0.41, atol=0.05).all()
        True
        """
        return ' '.join(np.random.choice(self.mdl.index, size=M, replace=True, p=self.mdl))

In [52]:
tokens = tokenize(beowulf)
unigram = UnigramLM(tokens)
unigram.sample(100)

In [ ]:
grader.check("q4")

### Conclusion: Baseline models

You've now trained two baseline language models capable of generating new text from a given training text. Attempt to answer these questions for yourself before you continue.

* Which model do you think is better? Why?
* What are the ways in which both of these models are bad?

## N-gram Model

Now you will build an N-gram language model, which we be a little more involved. 

**Before continuing, go back and read the introduction to understand the concepts described below.**

The `NGramLM` class contains a few extra methods and attributes beyond those of `UniformLM` and `UnigramLM`:

1. Instantiating `NGramLM` requires both a list of tokens and an `N > 0`, specifying the `N` in N-grams. This parameter is stored in an attribute `N`.
1. The `NGramLM` class has a method `create_ngrams` that takes in a list of tokens and returns a list of N-grams (an N-gram is a `tuple` of length `N`). This list of N-grams is then passed to the `train` method to train the N-gram model.
1. The `NGramLM` class has an attribute `mdl` that stores all the n-gram language models for `n=1...N`. The smaller n-grams are needed to compute the likelihood a word occurs at the start of a text. This code is included for you in the constructor.
1. While the `train` method still creates a language-model (in this case, an N-gram model), this model is most naturally stored as a DataFrame. This DataFrame will have three columns:
    - `'ngram'`: containing the n-grams found in the text.
    - `'n1gram'`: containing the (n-1)-grams upon which the n-grams in `ngram` are built.
    - `'prob'`: containing the probabilities of each n-gram in `ngram`.

### Creating n-grams

**Question 5.a**

When computing the likelihood that a word occurs in an N-gram model, you compute the probability that word occurs conditional on the (N-1) words that came before it. Therefore, you need to compute a list of N-grams out of your list of tokens (see the introduction for the definition).

Create a method of `NGramLM` called `create_ngrams` that takes in a list of tokens and returns a list of N-grams.

### Creating the N-gram LM

**Question 5.b**

Now, you will compute the probabilities that define N-gram language model itself. Recall that the N-gram LM consists of probabilities
$$P(w_1,\ldots,w_N) = P(w_N | w_1,\ldots, w_{N-1}) = \frac{C(w_1, \ldots, w_N)}{C(w_1,\ldots, w_{N-1})}$$

for every n-gram $(w_1, \ldots, w_N)$ that occurs in the text.

Create a method of `NGramLM` called `train` that takes in a list of N-grams and returns a dataframe of conditional probabilities with the following columns:

* `ngram` contains each N-gram that occurs in the text,
* `n1gram` contains the (N-1) tokens that precede the last token in the N-gram occupying the same row.
* `prob` contains the conditional probabilities associated to the N-gram occupying the same row.

### Computing probabilities from the N-gram LM

**Question 5.c**

Create a method of `NGramLM` called `probability` that takes in a 'sentence' (sequence of tokens) and returns the likelihood of the sentence under the language model (see the introduction of the formulas).

*Remark:* What is the probability of a sentence that contains a 'never before seen' word? What about a 'never before seen' (N-1)-gram? Is this reasonable? (Answer: no, it's not reasonable. Fixing this sort of deficiency is a topic in NLP).

### Sampling from the N-gram LM

**Question 5.d**

As you saw creating the baseline models, sampling from your trained LM provides a way to generate new language. You will code up a `sample` method for `NGramLM` and observe how much better (or worse) the generated language appears when compared to your baseline models.

Create a method of `NGramLM` called `sample` that takes in a number `M > 0` and generates a string of M tokens using the language model. It should begin with a starting token $\texttt{\\x02}$, then generate the following words from the probabilities in `self.mdl` and continue picking words conditional on the previous choice. The first $\texttt{\\x02}$ token should not count toward the total length. Helper functions and recursion will be very helpful.

*Remark:* If you find, at any point, your LM has no words to sample, then you should return the STOP token (`\x03`) and continue until you reach the required length.

*(Fun) Remark:* Try generating text using different values of `N`. What are the differences (qualitatively)? Don't be alarmed if your samples generate strange quasi-coherent sentences; it's just randomness!

*(Fun) Remark:* Try sampling from an `N`-gram model built on books written in different languages (It doesn't even need to be written using the Latin alphabet!)

**Question 5:**
Complete all four parts by completing the four methods in `NGramLM`

In [68]:
ngram = NGramLM(3, tokens)

In [69]:
print(ngram.sample(50))

In [ ]:
class NGramLM(object):

    def __init__(self, N, tokens):
        """
        Initializes a N-gram languange model using a
        list of tokens. It trains the language model
        using `train` and saves it to an attribute
        self.mdl.
        """

        self.N = N

        ngrams = self.create_ngrams(tokens)

        self.ngrams = ngrams
        self.mdl = self.train(ngrams)

        if N < 2:
            raise Exception('N must be greater than 1')
        elif N == 2:
            self.prev_mdl = UnigramLM(tokens)
        else:
            mdl = NGramLM(N-1, tokens)
            # self.prev_mdl = mdl.prev_mdl
            self.prev_mdl = mdl

    def create_ngrams(self, tokens):
        """
        create_ngrams takes in a list of tokens and returns a list of N-grams.
        The START/STOP tokens in the N-grams should be handled as
        explained in the notebook.

        :Example:
        >>> tokens = tuple('\x02 one two three one four \x03'.split())
        >>> bigrams = NGramLM(2, [])
        >>> out = bigrams.create_ngrams(tokens)
        >>> isinstance(out[0], tuple)
        True
        >>> out[0]
        ('\\x02', 'one')
        >>> out[2]
        ('two', 'three')
        """
        ngram_lst = []
        for i in range(len(tokens)-self.N+1):
            ngram_lst.append(tuple([tokens[i+j] for j in range(self.N)]))
        return ngram_lst

    def train(self, ngrams):
        """
        Trains a n-gram language model given a list of tokens.
        The output is a dataframe with three columns (ngram, n1gram, prob).

        :Example:
        >>> tokens = tuple('\x02 one two three one four \x03'.split())
        >>> bigrams = NGramLM(2, tokens)
        >>> set(bigrams.mdl.columns) == set('ngram n1gram prob'.split())
        True
        >>> bigrams.mdl.shape == (6, 3)
        True
        >>> bigrams.mdl['prob'].min() == 0.5
        True
        """
        if len(ngrams) == 0:
            return pd.DataFrame()
        # ngram counts C(w_1, ..., w_n)
        unique_ngrams_count = pd.Series(ngrams).value_counts()
        unique_ngrams = unique_ngrams_count.index.to_series()
        # n-1 gram counts C(w_1, ..., w_(n-1))
        unique_n1grams_count = pd.Series([ngram[:-1] for ngram in ngrams]).value_counts()

        df = pd.DataFrame({
            'ngram': unique_ngrams,
            'n1gram': unique_ngrams.apply(lambda x: x[:-1]),
            })

        # Create the conditional probabilities
        probability = df.apply(
            lambda x: unique_ngrams_count[x['ngram']]/unique_n1grams_count[x['n1gram']], axis=1)


        # Put it all together
        df = df.assign(prob=probability).reset_index(drop=True)

        return df
        ...
        ...

    def probability(self, words):
        """
        probability gives the probabiliy a sequence of words
        appears under the language model.
        :param: words: a tuple of tokens
        :returns: the probability `words` appears under the language
        model.

        :Example:
        >>> tokens = tuple('\x02 one two one three one two \x03'.split())
        >>> bigrams = NGramLM(2, tokens)
        >>> p = bigrams.probability('two one three'.split())
        >>> np.isclose(p, (1/4)*(1/2)*(1/3))
        True
        >>> bigrams.probability('one two five'.split()) == 0
        True
        """
        ngrams = pd.DataFrame({'ngram':self.create_ngrams(words)})
        merged_df = pd.merge(ngrams, self.mdl, how='left', on='ngram').fillna(0)
        prob = merged_df['prob'].prod()

        return prob * self.prev_mdl.probability(words[:self.N-1])


    def sample(self, M):
        """
        sample selects tokens from the language model of length M, returning
        a string of tokens.

        :Example:
        >>> tokens = tuple('\x02 one two three one four \x03'.split())
        >>> bigrams = NGramLM(2, tokens)
        >>> samp = bigrams.sample(3)
        >>> len(samp.split()) == 4  # don't count the initial START token.
        True
        >>> samp[:2] == '\\x02 '
        True
        >>> set(samp.split()) <= {'\\x02', '\\x03', 'one', 'two', 'three', 'four'}
        True
        """
        # Use a helper function to generate sample tokens of length `length`
        def get_tokens(self, M):
            if M == 0:
                word = ('\x02',)
            else:
                if M <= self.N-1:
                    prev_tokens = get_tokens(self.prev_mdl, M-1)

                    word_lst = self.mdl[self.mdl['n1gram']==prev_tokens]
                    if word_lst.shape[0] == 0:
                        word = prev_tokens + tuple('\x03')
                    else:
                        word = np.random.choice(word_lst['ngram'], size=1, replace=True, p=word_lst['prob'])[0]
                else:
                    prev_tokens = get_tokens(self, M-1)
                    word_lst = self.mdl[self.mdl['n1gram']==prev_tokens[-self.N+1:]]
                    if word_lst.shape[0]== 0:
                        word = prev_tokens + tuple('\x03')
                    else:
                        word = np.random.choice(word_lst['ngram'], size=1, replace=True, p=word_lst['prob'])[0]
                        word = prev_tokens[:-self.N+1] + word
            return word

        tokens = get_tokens(self, M)
        # Transform the tokens to strings
        string = ' '.join(list(tokens))
        return string

In [ ]:
grader.check("q5")

### Predicting the next word in a sentence

**OPTIONAL QUESTION**

Build a predictor that predicts the most likely word to follow a given sentence. The predictions will be the *maximum likelihood estimate* given by your N-gram model. Recall that your N-gram model contains the probabilities that a token follows an (n-1)-gram; your predictor will pick the token with the highest probability of occurring. 

Create a function that takes in an instantiated `NGramLM` object and a list of tokens, and predicts the *most likely token* to follow the list of tokens in the input (according to `NGramLM`).

*Remark:* For which N does this predictor work best? (Determine this by getting a feel for the output, we won't precisely answer this question).

*Remark:* What would this function look like if it were to take in `UniformLM` or `UnigramLM`?

## Model Performance Evaluation

**OPTIONAL QUESTION**

This question attempts to answer the question "which choice of `N` is best when creating an N-gram model?"

As you likely noticed when generating text using your N-gram models, the sentences appear to become more coherent as N increases. The naive conclusion would be to make N very large. However, such a choice has a downside that you will investigate.

One concern is that as N gets larger, the distribution of N-grams get more *sparse* -- i.e. the collection of n-grams all have counts close to 1. This makes the language model more likely to merely generate text that was already present in the original text, as opposed to generating *new* language (why?). 

In this question, you will quantify this observation: given an N-gram model and a sample of length M, what percentage of (N+1)-grams in the sample are present in the text that was used to create the model?

Create a function which takes in a list of tokens and:
1. Generates N-gram models for `N=2,3,4,5,6,7,8` from the given list of tokens,
2. Creates samples of length 1000 from each model (remove START/STOP tokens from the samples),
3. Calculates the proportion of (N+1)-grams in each sample that were already present in the original list of tokens.


# Congratulations, you finished the project!

### Before you submit:
* Be sure you run the doctests on all your code in `project.py`

### To submit:
* **Upload the .py file to gradescope**

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()